# Inspecting Lidar-derived Vector and Raster Products
We'll plot some of the layers we generated to see if there are any glaring errors that need to be resolved before archiving the project or cleaning up intermediate files.

In [ ]:
import os
import glob
import geopandas as gpd
from matplotlib import pyplot as plt
import rasterio
import random
from pyFIRS.utils import fname

In [ ]:
# where the imported lidar data is currently stored
WORKDIR = os.path.abspath('/storage/lidar/olympic-peninsula_2005/')

# where our derivative lidar products live
PROCESSED = os.path.join(WORKDIR,'processed')

## Take a look at the vector products
We'll get an overview of the lidar footprint, the tiles in it, and the building footprints.

In [ ]:
# inspect merged vectors
VECTORS = os.path.join(PROCESSED, 'vectors')
tiles = gpd.read_file(os.path.join(VECTORS, 'tiles.shp'))
foot = gpd.read_file(os.path.join(VECTORS, 'footprint.shp'))

fig, axs = plt.subplots(figsize=(20,20), sharex=True, sharey=True)
tiles.plot(ax=axs, edgecolor='gray')
foot.plot(ax=axs, facecolor='none', edgecolor='k')

plt.show();

Inspect a few of the building tiles closer-up.

In [ ]:
# inspect merged vectors
bld_tiles = glob.glob(os.path.join(VECTORS, 'building_tiles', '*.shp'))
samples = random.sample(bld_tiles, 5)

fig, axs = plt.subplots(5, 1, figsize=(20,20))
for i, sample in enumerate(samples):
    gdf = gpd.read_file(sample)
    gdf['dist_from_zero'] = (gdf.centroid.x**2 + gdf.centroid.y**2)**0.5
    gdf = gdf.sort_values(by=['dist_from_zero'])
    gdf.iloc[0:15].plot(ax=axs.ravel()[i])

plt.show();

## Take a look at the raster products
The only merged raster products are usually the layers generated from FUSION's gridmetrics and gridsurfacestats, which are low-enough resolution (e.g., 10m) to fit into a single GeoTiff. Let's look at those here.

In [ ]:
# inspect merged rasters
rasters = glob.glob(os.path.join(PROCESSED, 'rasters', '*.tif'))
num_rasters = len(rasters)
fig, axs = plt.subplots(num_rasters, 1, figsize=(20,20*num_rasters))
for i, raster in enumerate(rasters):
    with rasterio.open(raster) as src:
        img = src.read(1, masked=True)
    axs.ravel()[i].imshow(img)
    axs.ravel()[i].set_title(fname(raster))
plt.tight_layout();

To view the DEMs, hillshades, intensity rasters, and canopy height models that were probably generated at much higher resolution, we'll inspect a few random tiles.

In [ ]:
# inspect a few tiles
tiles = [fname(tile) for tile in glob.glob(os.path.join(PROCESSED, 'points', '*.laz'))]
samples = random.sample(tiles, 3)

RASTERS = os.path.join(PROCESSED, 'rasters')
dems = [os.path.join(RASTERS, 'dem_tiles', tile+'.tif') for tile in samples]
hills = [os.path.join(RASTERS, 'hillshade_tiles', tile+'.tif') for tile in samples]
chms = [os.path.join(RASTERS, 'chm_tiles', tile+'.tif') for tile in samples]
intens = [os.path.join(RASTERS, 'intensity_tiles', tile+'.tif') for tile in samples]
plots = dems + hills + chms + intens

fig, axs = plt.subplots(4, 3, figsize=(12,18), sharex='row', sharey='row')
for i, sample in enumerate(plots):
    with rasterio.open(sample) as src:
        img = src.read(1, masked=True)
        axs.ravel()[i].imshow(img, cmap='gray')

plt.show();